# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

In [2]:
print("https://maps.googleapis.com/maps/api/place/findplacefromtext/json")

https://maps.googleapis.com/maps/api/place/findplacefromtext/json


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
load_csv = "weatherpy.csv"
city_df = pd.read_csv(load_csv)

city_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Port Elizabeth,40,ZA,1613943171,72,-33.9180,25.5701,64.40,11.50
1,1,Rikitea,0,PF,1613943172,68,-23.1203,-134.9692,79.81,10.02
2,2,Nueva Guinea,20,NI,1613943238,75,11.6876,-84.4562,77.13,13.89
3,3,Tasiilaq,90,GL,1613943238,93,65.6145,-37.6368,32.00,36.82
4,4,Ushuaia,90,AR,1613943010,70,-54.8000,-68.3000,44.60,1.32


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
city_df = city_df.dropna()
humidity = city_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = city_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = city_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 0]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 60]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] >= 20]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] >= 30]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Humidity'] <= 70]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,279,279,Torbay,90,CA,1613947041,61,47.6666,-52.7314,12.0,21.85
1,471,471,Lloydminster,90,CA,1613948519,52,53.2835,-110.0016,41.0,20.71


Addded humidity and differing temperatures as I hate the heat and the humidity!

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan

hotel_df = hotel_df.drop(columns=['index'])
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,279,Torbay,90,CA,1613947041,61,47.6666,-52.7314,12.0,21.85,NaN
1,471,Lloydminster,90,CA,1613948519,52,53.2835,-110.0016,41.0,20.71,NaN


In [7]:
Hotels = []
results = []

hotel_df = narrowed_city_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

print("Begining Data Retrieval")
print(f"-------------------------------")

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
   
    results = response['results']
        
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.dropna()
        
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Begining Data Retrieval
-------------------------------
Retrieving Results for Index 0: Torbay.
Closest hotel is Gallows Cove.
------------
Retrieving Results for Index 1: Lloydminster.
Closest hotel is BCMInns - Lloydminster.
------------


In [8]:
hotel_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,279,279,Torbay,90,CA,1613947041,61,47.6666,-52.7314,12.0,21.85,Gallows Cove
1,471,471,Lloydminster,90,CA,1613948519,52,53.2835,-110.0016,41.0,20.71,BCMInns - Lloydminster


In [9]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig




Figure(layout=FigureLayout(height='420px'))